# Playground for analysis of Mitsuba vs. reference data

Chowdhary et al. "Testbed results for scalar and vector radiative transfer computations of light in atmosphere-ocean systems." 2020.

Models:
- AOS 1: Rayleigh atmosphere, rough ocean surface, [no ocean body]
- AOS 2: [no atmosphere], rough ocean surface, Rayleigh ocean body
- AOS 3: Rayleigh atmosphere, rough ocean surface, Rayleigh ocean body
- AOS 4: Rayleigh atmosphere, rough ocean surface, ocean body with tabulated/Mie-Rayleigh blended phase function

---
Zhai. Successive orders of scattering method used in same "Testbed results" paper as above.

Models:
- AOS 3: Rayleigh atmosphere, *smooth* ocean surface, Rayleigh ocean body


---
Natraj et al. "Rayleigh Scattering in Planetary Atmospheres: Corrected Tables through Accurate Computation of X and Y Functions" 2009.

Models:
- AOS 1*: Rayleigh atmosphere (variable optical depth tau), black ocean surface

---
When data is not available for a particular set of parameters, relative error will be 100% for all sample counts.

Written by Kate 07/11

In [1]:
import sys
sys.path.append('..')

import math
import json
import numpy as np
# import h5py

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import widgets, interact

from parse_egap_data import get_data as get_data_ch
from parse_sos_data import get_data as get_data_zh

In [2]:
# Chowdhary et al. data
models_ch  = ['AOS_1', 'AOS_2', 'AOS_3', 'AOS_4']
λ_ch       = np.array([350., 450., 550., 650.])
θp_ch      = np.array([30.0, 60.0])
θs_ch      = np.linspace(0.0, 60.0, 13)
φ_ch       = np.array([0.0, 60.0, 180.0, 240.0])

# Available Mitsuba data in direct comparison to Chowdhary et al.
models_ch_ = ['AOS_1', 'AOS_2', 'AOS_3']
λ_ch_      = np.array([350., 450., 550., 650.])
θp_ch_     = np.array([30.0, 60.0])
# θs_ch_     = np.linspace(0.0, 60.0, 13)
φ_ch_      = np.array([0.0, 60.0, 180.0, 240.0])

# Where to pull Mitsuba data from
data_dir_ch = ['mitsuba_data/chowdhary/']
data_dir_zh = ['mitsuba_data/zhai/']

θs_ch_ = {}
θs_ch_['mitsuba_data/chowdhary/'] = np.linspace(0.0, 60.0, 7)
θs_ch_['mitsuba_data/zhai/'] = np.linspace(0.0, 60.0, 13)
θs_ch_['default'] = np.linspace(0.0, 60.0, 13)

from parse_natraj_data import parse_file

# Natraj et al. data
models_na_ = ['AOS_1*']
τ_na       = [0.02, 0.05, 0.1, 0.15, 0.25, 0.5, 1.0]
ρ_na       = [0.0, 0.25, 0.8]  # albedo
μp_na      = [0.1, 0.2, 0.4, 0.6, 0.8, 0.92, 1.0]
μs_na      = [0.02, 0.06, 0.1, 0.16, 0.2, 0.28, 0.32, 0.4, 0.52, 0.64, 0.72, 0.84, 0.92, 0.96, 0.98, 1.0]
φ_na       = [0.0, 30.0, 60.0, 90.0, 120.0, 150.0, 180.0]

file_prefix_na = '../natraj_data/'

def create_filename(stokes_str, tau):
    tau_str = str(tau) if tau != 1.0 else '1'
    return stokes_str + '_UP_TAU_' + tau_str

data_ref_na = {'I': {}, 'Q': {}, 'U': {}}

for s in ['I', 'Q', 'U']:
    for t in τ_na:
        filename = create_filename(s, t)
        chunk = parse_file(file_prefix_na + filename)
        data_ref_na[s].update(chunk)

# Available Mitsuba data in direct comparison to Natraj et al.
models_na_ = ['AOS_1*']
τ_na_      = np.array([0.02, 0.1, 0.5, 1.0])
μp_na_     = np.array([0.1, 0.6, 1.0])
μs_na_     = np.array([0.02, 0.1, 0.2, 0.32, 0.52, 0.72, 0.92, 1.0])
φ_na_      = np.array([0.0])

data_dir_na = ['mitsuba_data/natraj/']

# Number of samples
n_samples = {}
n_samples['mitsuba_data/chowdhary/'] = np.array([4, 32, 256, 2048]) * 512 * 512
n_samples['mitsuba_data/zhai/'] = np.array([4, 32, 256, 2048]) * 512 * 512
n_samples['mitsuba_data/natraj/'] = np.array([4, 32, 256, 2048]) * 512 * 512
n_samples['default'] = np.array([1e3, 1e4, 1e5, 1e6]) * 32 * 32

In [3]:
# Build UI above plots

def selection_slider(options, value, description, disabled = False):
    return widgets.SelectionSlider(options = options, value = value, description = description, 
                                   continuous_update = False, orientation = 'horizontal', readout = True, 
                                   disabled = disabled, layout = widgets.Layout(width = '95%'))

data_select_label  = widgets.Label('mitsuba data 1')
data_select        = widgets.Dropdown(options = data_dir_ch, description = '', disabled = False) 
data_select_label2 = widgets.Label('mitsuba data 2')
data_select2       = widgets.Dropdown(options = ['None'] + data_dir_ch, description = '', disabled = False)
data_select_box    = widgets.HBox(children = [data_select_label, data_select])
data_select_box2   = widgets.HBox(children = [data_select_label2, data_select2])

error_select  = widgets.ToggleButtons(options = ['error', 'variance'], description = 'data type', disabled = False)
model_select  = widgets.ToggleButtons(options = models_ch_, description = 'model', disabled = False) 
stokes_select = widgets.ToggleButtons(options = ['I', 'Q', 'U', 'DOP'], description = 'stokes', disabled = False,
                    tooltips = ['Total radiance', 
                                '0/90 deg polarized component', 
                                '45/135 deg polarized component',
                                'Degree of polarization'],
                )

# Chowdhary et al. parameters
λ_ch_ui  = selection_slider(options = λ_ch_, value = 350.0, description = 'λ')
θp_ch_ui = selection_slider(options = θp_ch_, value = 30.0, description = 'θp')
θs_ch_ui = selection_slider(options = θs_ch_['default'], value = 30.0, description = 'θs')
φ_ch_ui  = selection_slider(options = φ_ch_, value = 0.0, description = 'φ')

# Natraj et al. parameters
τ_na_ui  = selection_slider(options = τ_na_, value = 0.5, description = 'τ')
μp_na_ui = selection_slider(options = μp_na_, value = 0.6, description = 'μp')
μs_na_ui = selection_slider(options = μs_na_, value = 0.1, description = 'μs')
φ_na_ui  = selection_slider(options = φ_na_, value = 0.0, description = 'φ')

# Chowdhary or Natraj
source_label = widgets.Label('source')
source_ch    = widgets.Button(description = 'chowdhary', disabled = False, button_style = '')
source_zh    = widgets.Button(description = 'zhai', disabled = False, button_style = '')
source_na    = widgets.Button(description = 'natraj', disabled = False, button_style = '')
source_selected = widgets.Label(value = 'chowdhary')

source_options = widgets.HBox(children = [source_label, source_ch, source_zh, source_na])

ui_const = [source_options, data_select_box, data_select_box2, error_select, stokes_select, model_select]
ui_ch    = [λ_ch_ui, θp_ch_ui, θs_ch_ui, φ_ch_ui]
ui_na    = [τ_na_ui, μp_na_ui, μs_na_ui, φ_na_ui]

ui = widgets.VBox(children = ui_const + ui_ch)

In [4]:
def stokes2idx(stokes_str):
    if stokes_str == 'I':
        idx = 0
    elif stokes_str == 'Q':
        idx = 1
    elif stokes_str == 'U':
        idx = 2
    elif stokes_str == 'V':
        idx = 3
        
    return idx

# Get Chowdhary et al. reference data
def get_ref_ch_stokes(model, stokes, λ, θp, θs, φ):
    ts  = np.where(θs == θs_ch)[0][0]
    s   = stokes2idx(stokes) + 1 # +1 since for this dataset the first column is θs angle
    
    data_tmp = get_data_ch('../egap_data', model, θp, φ, λ)
    
    return data_tmp[:,s][ts]
        
# Get Mitsuba data relating to Chowdhary et al. parameters
def get_mts_ch_stokes(data_select, model_select, stokes, λ, θp, θs, φ):
    model_file = '../' + data_select + model_select + '_mts.npy'
    data = np.load(model_file)
    
    ts_tmp = np.where(θs == θs_ch_[data_select])[0]
    if len(ts_tmp) == 0:
        return np.zeros(len(n_samples))
    
    w  = np.where(λ  == λ_ch_)[0][0]
    tp = np.where(θp == θp_ch_)[0][0]
    ts = ts_tmp[0]
    p  = np.where(φ  == φ_ch_)[0][0]
    s  = stokes2idx(stokes)
    
    data_tmp = data[w, tp, ts, p, :, s, :]
        
    # Convert to reflectance quantity to compare to Chowhdary et al.
    μp = np.cos(np.deg2rad(θp))
    data_tmp /= μp

    return data_tmp

# Get Zhai reference data
def get_ref_zh_stokes(model, stokes, λ, θp, θs, φ):
    ts  = np.where(θs == θs_ch)[0][0]
    s   = stokes2idx(stokes)
    
    data_tmp = get_data_zh('../sos_data', model, θp, φ, λ)
    
    return data_tmp[:,s][ts]
        
# Get Mitsuba data relating to Zhai et al. parameters
def get_mts_zh_stokes(data_select, model_select, stokes, λ, θp, θs, φ):
    model_file = '../' + data_select + model_select + '_mts.npy'
    data = np.load(model_file)
    
    ts_tmp = np.where(θs == θs_ch_[data_select])[0]
    if len(ts_tmp) == 0:
        return np.zeros(len(n_samples))
    
    w  = np.where(λ  == λ_ch_)[0][0]
    tp = np.where(θp == θp_ch_)[0][0]
    ts = ts_tmp[0]
    p  = np.where(φ  == φ_ch_)[0][0]
    s  = stokes2idx(stokes)
    
    data_tmp = data[w, tp, ts, p, :, s, :]
    
    # Do not need to convert to reflectance quantity since Zhai data is given in radiance
    return data_tmp

# Get Natraj et al. reference data
def get_ref_na_stokes(stokes, τ, μp, μs, φ):
    return data_ref_na[stokes][τ][0.0][μp][μs][φ]

# Get Mitsuba data relating to Natraj et al. parameters
def get_mts_na_stokes(data_select, model_select, stokes, τ, μp, μs, φ):
    model_file = '../' + data_select + model_select + '_mts.npy'
    data = np.load(model_file)
    
    t  = np.where(τ  == τ_na_)[0][0]
    ms = np.where(μs == μs_na_)[0][0]
    mp = np.where(μp == μp_na_)[0][0]
    p  = np.where(φ  == φ_na_)[0][0]
    s  = stokes2idx(stokes)

    return data[t, mp, ms, p, :, s, :]

# Degree of polarization
def dop(si, sq, su):
    qu_sq = sq * sq + su * su
    return np.sqrt(qu_sq) / si * 100.0

# Get Chowdhary et al. reference data and calculate DOP
def get_ref_ch_dop(model, λ, θp, θs, φ):
    ts  = np.where(θs == θs_ch)[0][0]
    
    data_tmp = get_data_ch('../egap_data', model, θp, φ, λ)
    
    si = data_tmp[:,1][ts]
    sq = data_tmp[:,2][ts]
    su = data_tmp[:,3][ts]
    
    return dop(si, sq, su)

# Get Mitsuba data relating to Chowdhary et al. parameters and calculate DOP
def get_mts_ch_dop(data_select, model_select, λ, θp, θs, φ):
    model_file = '../' + data_select + model_select + '_mts.npy'
    data = np.load(model_file)
        
    ts_tmp = np.where(θs == θs_ch_[data_select])[0]
    if len(ts_tmp) == 0:
        return np.zeros(len(n_samples))
    
    w  = np.where(λ  == λ_ch_)[0][0]
    tp = np.where(θp == θp_ch_)[0][0]
    ts = ts_tmp[0]
    p  = np.where(φ  == φ_ch_)[0][0]
    
    si = data[w, tp, ts, p, :, 0, :]
    sq = data[w, tp, ts, p, :, 1, :]
    su = data[w, tp, ts, p, :, 2, :]
        
    # Convert to reflectance quantity to compare to Chowhdary et al. and calculate DOP
    μp = np.cos(np.deg2rad(θp))
    si /= μp
    sq /= μp
    su /= μp

    return dop(si, sq, su)

# Get Zhai et al. reference data and calculate DOP
def get_ref_zh_dop(model, λ, θp, θs, φ):
    ts  = np.where(θs == θs_ch)[0][0]
    
    data_tmp = get_data_zh('../sos_data', model, θp, φ, λ)
    
    si = data_tmp[:,0][ts]
    sq = data_tmp[:,1][ts]
    su = data_tmp[:,2][ts]
    
    return dop(si, sq, su)

# Get Mitsuba data relating to Zhai parameters and calculate DOP
def get_mts_zh_dop(data_select, model_select, λ, θp, θs, φ):
    model_file = '../' + data_select + model_select + '_mts.npy'
    data = np.load(model_file)
        
    ts_tmp = np.where(θs == θs_ch_[data_select])[0]
    if len(ts_tmp) == 0:
        return np.zeros(len(n_samples))
    
    w  = np.where(λ  == λ_ch_)[0][0]
    tp = np.where(θp == θp_ch_)[0][0]
    ts = ts_tmp[0]
    p  = np.where(φ  == φ_ch_)[0][0]
    
    si = data[w, tp, ts, p, :, 0, :]
    sq = data[w, tp, ts, p, :, 1, :]
    su = data[w, tp, ts, p, :, 2, :]
        
    # Do not convert to reflectance quantity here
    return dop(si, sq, su)

# Get Natraj et al. reference data
def get_ref_na_dop(stokes, τ, μp, μs, φ):
    si = data_ref_na['I'][τ][0.0][μp][μs][φ]
    sq = data_ref_na['Q'][τ][0.0][μp][μs][φ]
    su = data_ref_na['U'][τ][0.0][μp][μs][φ]
    
    return dop(si, sq, su)

# Get Mitsuba data relating to Natraj et al. parameters and calculate DOP
def get_mts_na_dop(data_select, model_select, τ, μp, μs, φ):
    model_file = '../' + data_select + model_select + '_mts.npy'
    data = np.load(model_file)
    
    t  = np.where(τ  == τ_na_)[0][0]
    ms = np.where(μs == μs_na_)[0][0]
    mp = np.where(μp == μp_na_)[0][0]
    p  = np.where(φ  == φ_na_)[0][0]

    si = data[t, mp, ms, p, :, 0, :]
    sq = data[t, mp, ms, p, :, 1, :]
    su = data[t, mp, ms, p, :, 2, :]
    
    return dop(si, sq, su)

In [5]:
plots = []

def create_error_plot():
    return go.Scatter(x = n_samples['default'], y = np.zeros(len(n_samples['default'])),
                        mode = 'lines+markers', 
                        line = dict(),
                        name = '% Error',
                        error_y = dict(
                            type = 'data',
                            array = np.zeros(len(n_samples['default'])),
                            visible = True
                        )
                      )

plots.append(create_error_plot())
plots.append(create_error_plot())

layout = go.Layout(
#     showlegend = False,
    title = 'Convergence of Mitsuba simulation vs. reference (line = mean, bars = 1 std dev)',
    xaxis_title_text = 'Number of Samples',
    yaxis = dict(
                title_text = '% Error'
#                 range = [-1.0, 1.0]
            )
)

fig = go.FigureWidget(data = plots, layout = layout)
fig.update_xaxes(type = 'log')
fig.update_yaxes(type = 'log')

def update(source_selected, data_select, data_select2, stokes_select, model_select, error_select,
           λ_ch_ui, θp_ch_ui, θs_ch_ui, φ_ch_ui,
           τ_na_ui, μp_na_ui, μs_na_ui, φ_na_ui):
    
    plot_data2 = (data_select2 != 'None')

    # Load data
    if source_selected == 'chowdhary':
        if stokes_select == 'DOP':
            data_ref_new = get_ref_ch_dop(model_select, λ_ch_ui, θp_ch_ui, θs_ch_ui, φ_ch_ui)
            data_mts_new = get_mts_ch_dop(data_select, model_select, λ_ch_ui, θp_ch_ui, θs_ch_ui, φ_ch_ui)
        else:
            data_ref_new = get_ref_ch_stokes(model_select, stokes_select, λ_ch_ui, θp_ch_ui, θs_ch_ui, φ_ch_ui)
            data_mts_new = get_mts_ch_stokes(data_select, model_select, stokes_select, λ_ch_ui, θp_ch_ui, θs_ch_ui, φ_ch_ui)

        if plot_data2:
            if stokes_select == 'DOP':
                data_mts_new2 = get_mts_ch_dop(data_select2, model_select, λ_ch_ui, θp_ch_ui, θs_ch_ui, φ_ch_ui)
            else:
                data_mts_new2 = get_mts_ch_stokes(data_select2, model_select, stokes_select, λ_ch_ui, θp_ch_ui, θs_ch_ui, φ_ch_ui)

    elif source_selected == 'zhai':
        if stokes_select == 'DOP':
            data_ref_new = get_ref_zh_dop(model_select, λ_ch_ui, θp_ch_ui, θs_ch_ui, φ_ch_ui)
            data_mts_new = get_mts_zh_dop(data_select, model_select, λ_ch_ui, θp_ch_ui, θs_ch_ui, φ_ch_ui)
        else:
            data_ref_new = get_ref_zh_stokes(model_select, stokes_select, λ_ch_ui, θp_ch_ui, θs_ch_ui, φ_ch_ui)
            data_mts_new = get_mts_zh_stokes(data_select, model_select, stokes_select, λ_ch_ui, θp_ch_ui, θs_ch_ui, φ_ch_ui)

        if plot_data2:
            if stokes_select == 'DOP':
                data_mts_new2 = get_mts_zh_dop(data_select2, model_select, λ_ch_ui, θp_ch_ui, θs_ch_ui, φ_ch_ui)
            else:
                data_mts_new2 = get_mts_zh_stokes(data_select2, model_select, stokes_select, λ_ch_ui, θp_ch_ui, θs_ch_ui, φ_ch_ui)
        
    elif source_selected == 'natraj':
        if stokes_select == 'DOP':
            data_ref_new = get_ref_na_dop(model_select, τ_na_ui, μp_na_ui, μs_na_ui, φ_na_ui)
            data_mts_new = get_mts_na_dop(data_select, model_select, τ_na_ui, μp_na_ui, μs_na_ui, φ_na_ui)
        else:
            data_ref_new = get_ref_na_stokes(stokes_select, τ_na_ui, μp_na_ui, μs_na_ui, φ_na_ui)
            data_mts_new = get_mts_na_stokes(data_select, model_select, stokes_select, τ_na_ui, μp_na_ui, μs_na_ui, φ_na_ui)

        if plot_data2:
            if stokes_select == 'DOP':
                data_mts_new2 = get_mts_na_dop(data_select2, model_select, τ_na_ui, μp_na_ui, μs_na_ui, φ_na_ui)
            else:
                data_mts_new2 = get_mts_na_stokes(data_select2, model_select, stokes_select, τ_na_ui, μp_na_ui, μs_na_ui, φ_na_ui)

    x_samples = n_samples[data_select]
    n_trials = data_mts_new.shape[1]

    # Compute error
    data_ref_fill = np.tile(data_ref_new, (len(x_samples), n_trials))

    abs_error = np.abs(data_mts_new - data_ref_fill)
    abs_error_mean = np.mean(abs_error, axis = 1)
    abs_error_std  = np.std(abs_error, axis = 1)
    
    rel_error = np.abs((data_mts_new - data_ref_fill) / data_ref_fill * 100.0)
#     rel_error = (data_mts_new - data_ref_fill) / data_ref_fill * 100.0
    rel_error_mean = np.mean(rel_error, axis = 1)
    rel_error_std  = np.std(rel_error, axis = 1)
    
    variance = np.var(data_mts_new, axis = 1)
    
    if plot_data2:
        x_samples2 = n_samples[data_select2]
        n_trials2 = data_mts_new2.shape[1]

        # Compute error
        data_ref_fill2 = np.tile(data_ref_new, (len(x_samples2), n_trials2))

        abs_error2 = np.abs(data_mts_new2 - data_ref_fill2)
        abs_error_mean2 = np.mean(abs_error2, axis = 1)
        abs_error_std2  = np.std(abs_error2, axis = 1)
        
        rel_error2 = np.abs((data_mts_new2 - data_ref_fill2) / data_ref_fill2 * 100.0)
        rel_error_mean2 = np.mean(rel_error2, axis = 1)
        rel_error_std2  = np.std(rel_error2, axis = 1)
        
        variance2 = np.var(data_mts_new2, axis = 1)

    # Update plots
    with fig.batch_update():
        if error_select == 'error':
            if stokes_select == 'DOP' or not np.all(data_ref_fill): # np.all(): check for zeros / divide by zero
                fig.update_yaxes(title_text = 'Abs Error')
                title_text = '% Error'
                label = 'Abs Error (' + data_select + ')'
                y_data = abs_error_mean
                y_std  = abs_error_std

                if plot_data2:
                    label2 = 'Abs Error (' + data_select2 + ')'
                    y_data2 = abs_error_mean2
                    y_std2  = abs_error_std2

            else:
                fig.update_yaxes(title_text = '% Error')
                label = '% Error (' + data_select + ')'
                y_data = rel_error_mean
                y_std  = rel_error_std

                if plot_data2:
                    label2 = '% Error (' + data_select2 + ')'
                    y_data2 = rel_error_mean2
                    y_std2  = rel_error_std2
        else:
            fig.update_yaxes(title_text = 'Variance')
            label = 'Variance (' + data_select + ')'
            y_data = variance
            y_std = np.zeros_like(y_data)
            
            if plot_data2:
                    label2 = 'Variance (' + data_select2 + ')'
                    y_data2 = variance2
                    y_std2  = np.zeros_like(y_data2)
                    
        
        fig.data[0].name = label
        fig.data[0].x = x_samples
        fig.data[0].y = y_data
        fig.data[0].error_y.array = y_std
        
        if plot_data2:
            fig.data[1].name = label2
            fig.data[1].x = x_samples2
            fig.data[1].y = y_data2
            fig.data[1].error_y.array = y_std2
        else:
            fig.data[1].name = ''
            fig.data[1].x = n_samples
            fig.data[1].y = np.zeros(len(n_samples))
            fig.data[1].error_y.array = np.zeros(len(n_samples))
        
out = widgets.interactive_output(update, {'source_selected': source_selected,
                                          'data_select': data_select,
                                          'data_select2': data_select2,
                                          'stokes_select': stokes_select, 
                                          'model_select': model_select,
                                          'error_select': error_select,
                                          'λ_ch_ui': λ_ch_ui, 
                                          'θp_ch_ui': θp_ch_ui, 
                                          'θs_ch_ui': θs_ch_ui, 
                                          'φ_ch_ui': φ_ch_ui,
                                          'τ_na_ui': τ_na_ui, 
                                          'μp_na_ui': μp_na_ui, 
                                          'μs_na_ui': μs_na_ui, 
                                          'φ_na_ui': φ_na_ui})

def source_ch_click(_):
    data_select.options = data_dir_ch
    data_select2.options = ['None'] + data_dir_ch
    model_select.options = models_ch_
    
    ui.children = ui_const + ui_ch
    source_selected.value = 'chowdhary'
    
def source_na_click(_):        
    data_select.options = data_dir_na
    data_select2.options = ['None'] + data_dir_na
    model_select.options = models_na_
    
    ui.children = ui_const + ui_na
    source_selected.value = 'natraj'
    
def source_zh_click(_):        
    data_select.options = data_dir_zh
    data_select2.options = ['None'] + data_dir_zh
    model_select.options = models_ch_
    
    ui.children = ui_const + ui_ch
    source_selected.value = 'zhai'
    
source_ch.on_click(source_ch_click)
source_na.on_click(source_na_click)
source_zh.on_click(source_zh_click)

display(ui)
fig

FigureWidget({
    'data': [{'error_y': {'array': array([0.07791082, 0.0380801 , 0.01287988, 0.00407401]),
   …